In [1]:
import cv2                                # state of the art computer vision algorithms library
import numpy as np                        # fundamental package for scientific computing
import matplotlib.pyplot as plt           # 2D plotting library producing publication quality figures
import pyrealsense2 as rs                 # Intel RealSense cross-platform open-source API
# import pyntcloud
# from pyntcloud import PyntCloud
import scipy
from sklearn.neighbors import NearestNeighbors
import itertools
from scipy import linalg
import matplotlib as mpl
import src.realsense.dataset as rld
import src.realsense.camera as rlc
# import src.visual3d.geometry as geom
import src.mymodel.gmm as mygmm
import torch

import rosbag
from scipy import linalg
from sklearn import mixture
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

from matplotlib import animation, rc
from IPython.display import HTML
import matplotlib
matplotlib.use("Agg")
import matplotlib.animation as manimation
from IPython.display import Video, clear_output
import time
# from sklearn import linear_model

from detect_pose_ros import get_pose

print("Environment Ready")

# %matplotlib
%load_ext autoreload
%autoreload 2

Environment Ready


In [2]:
# frame from bags
bag_name = 'single_layer'
np.random.seed(0)
pipe = rs.pipeline()
cfg = rs.config()
cfg.enable_device_from_file(f"data/{bag_name}.bag")
queue = rs.frame_queue(100)
profile = pipe.start(cfg)
playback = profile.get_device().as_playback()
playback.set_real_time(False)
# video writter

FFMpegWriter = manimation.writers['ffmpeg']
metadata = dict(title='Movie Test', artist='Matplotlib',
                comment='Movie support!')
writer = FFMpegWriter(fps=10, metadata=metadata)

In [3]:
for a in range(0):
    pipe.wait_for_frames()

# get intrinsics
bag = rosbag.Bag(f'data/{bag_name}.bag')
count_loop = 0
for topic, msg, t in bag.read_messages():
    if 'Color_0/info/camera_info' in topic:
        K = msg.K
    count_loop +=1
    if count_loop > 20:
        break
bag.close()
K = np.array(K).reshape(3,3)
        
fig, axarr = plt.subplots(2,2,figsize=(11,6))
axarr = axarr.flatten()
total_frames = 1000


In [4]:
start_time = time.time()
with writer.saving(fig, f"results/{bag_name}.mp4", 100):
    for frame_idx in range(total_frames):
        start_readframe_time = time.time()
        # Store next frameset for later processing:
        frameset = pipe.wait_for_frames()
        color_frame = frameset.get_color_frame()
        depth_frame = frameset.get_depth_frame()

        # visualize the capture frame
        color = np.asanyarray(color_frame.get_data())
        colorizer = rs.colorizer()
        colorized_depth = np.asanyarray(colorizer.colorize(depth_frame).get_data())
        H,W = color.shape[:2]
        gray = np.mean(colorized_depth,axis=2).astype(np.uint8)

        # align rgb and depth image
        align = rs.align(rs.stream.color)
        frameset = align.process(frameset)

        # Update color and depth frames:
        aligned_depth_frame = frameset.get_depth_frame()
        colorized_depth = np.asanyarray(colorizer.colorize(aligned_depth_frame).get_data())
        depth_scale = profile.get_device().first_depth_sensor().get_depth_scale()
        depth = np.asanyarray(aligned_depth_frame.get_data())
        depth = depth*depth_scale
        end_readframe_time = time.time()
        
        
        # get pose
        
        intersections, vectors1, vectors2 = get_pose(color,depth,K)
        
        
#         # # visualize 
#         # 1 - filtered
#         axarr[0].clear()
#         axarr[0].imshow(color_top_full)
#         axarr[0].title.set_text('slicing out top layer')

#         # 2 - initial GMM
#         axarr[1].clear()
#         axarr[1].imshow(depth_top,cmap = "Greys")
#         my_gmm.plot_results(X,gmm_list='original',ax = axarr[1])
#         axarr[1].title.set_text('initial GMM')
#         axarr[1].invert_yaxis()


#         3 - filter out fat
#         axarr[2].clear()
#         axarr[2].imshow(depth_top,cmap="Greys")
#         my_gmm.filter_GMM(X,filter_ratio=4) # change accordingly
#         my_gmm.merge_GMM(X)
#         my_gmm.filter_list = my_gmm.merge_list
#         my_gmm.plot_results(X,gmm_list='merge_list',ax = axarr[2])
#         axarr[2].invert_yaxis()
#         axarr[2].title.set_text('filter out fat GMM')

#         4 - generate pose
        axarr[3].clear()
        axarr[3].imshow(color)
        if len(intersections)!=0:
            axarr[3].scatter(intersections[:,0],intersections[:,1],100)
        for (px,py),(v1x,v1y),(v2x,v2y) in zip(intersections,vectors1,vectors2):
            vlength = 100
            if v1x<0:
                v1x = -v1x; v1y=-v1y
            if v2x<0:
                v2x = -v2x; v2y = -v2y
            axarr[3].arrow(px,py,vlength*v1x,vlength*v1y,head_width=30)
            axarr[3].arrow(px,py,vlength*v2x,vlength*v2y,head_width=30)       
            
        axarr[3].title.set_text('final vector')
        axarr[3].set_xlim(0,1280); axarr[3].set_ylim(0,720)
        axarr[3].invert_yaxis()

        fig.suptitle(f'frame = {frame_idx}/{total_frames}')
        writer.grab_frame()
        
        # progress bar
        clear_output(wait=True)
#         print(f"plane segmentation time = {end_plane_seg_time - start_plane_seg_time:.4f} s, GMM time = {end_gmm_time-start_gmm_time:.4f} s")
#         print(f"linear regression time = {end_linear_regression_time - start_linear_regression_time:.4f} s")
#         print(f"forward projection time = {end_projection_time-start_projection_time:.4f} s, back projection time = {end_backproj_time - start_backproj_time:.4f} s") 
#         print(f"prepare_gmm_time = {end_prepare_gmm_time - start_prepare_gmm_time:.4f} s")
#         print(f"readframe_time = {end_readframe_time-start_readframe_time:.4f} s")
#         print(f"saving the {frame_idx}/{total_frames} frame")
        elapsed_time = time.time() - start_time
        bar_width= 80
        b = int(bar_width * frame_idx / total_frames)
        l = bar_width - b
        print(u"\u2588" * int(b) + '-' * int(l),f'{frame_idx/total_frames*100:.2f}%')
        print(f"elapsed time = {elapsed_time/60:.2f} min, time remain = {elapsed_time/(1+frame_idx)*(total_frames-frame_idx)/60:.2f} min")
        
        

███████████████████████████████████████████████████████████████████████████████- 99.90%
elapsed time = 11.85 min, time remain = 0.01 min


In [5]:
Video(f"results/{bag_name}.mp4")